In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import warnings 
warnings.filterwarnings('ignore')

<h2>Load the dataset</h2>

In [2]:
gt = pd.read_csv('dataset\\Global_terrorist.csv', encoding='ISO-8859-1')

In [3]:
gt.head(5)

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


<h2>Filtering the data.</h2>

In [4]:
f1 = gt[gt['region_txt'] == "Middle East & North Africa"]
f2 = f1[f1['iyear'] >= 2000]
f3 = f2[f2['attacktype1_txt'] >= 'Bombing/Explosion']
filter4 = f3[f3['nkill'] >= 10]

<h2>Data aggregation.</h2>

In [5]:
spatial_data = filter4.groupby(['longitude', 'latitude']).size().reset_index(name='terror_count')
spatial_data['normalized_weight'] = (spatial_data['terror_count'] - spatial_data['terror_count'].min()) /\
                                    (spatial_data['terror_count'].max() - spatial_data['terror_count'].min())

In [6]:
spatial_data

,longitude,latitude,terror_count,normalized_weight
0,-8.000000,31.633333,1,0.000000
1,-7.610000,33.590000,1,0.000000
2,-0.408333,35.519444,1,0.000000
3,0.138188,35.391899,1,0.000000
4,0.719620,35.961568,1,0.000000
...,...,...,...,...
734,52.583610,29.591325,1,0.000000
735,60.582068,27.529991,1,0.000000
736,60.862701,29.483576,4,0.007481
737,61.446529,29.013931,1,0.000000


<h2>Preparing heatmap and the data.</h2>

In [7]:
pip install folium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import folium
from folium.plugins import HeatMap

In [9]:
spatial_list1 = spatial_data[['longitude', 'latitude', 'normalized_weight']].values.tolist()
base_map = folium.Map(location=[30, 40], zoom_start=4)
HeatMap(spatial_list1).add_to(base_map)
base_map.save('myGeoSpatial.html')
base_map

<h1>Regional Analysis</h1>

In [20]:
region_f1 = gt[(gt['region_txt'] == 'South America') & (gt['iyear'] >= 2000) & (gt['nkill'] >= 10)]

regional_spatial = region_f1.groupby(['longitude', 'latitude']).size().reset_index(name='terror_count')
regional_spatial['normalized_weight'] = (regional_spatial['terror_count'] - regional_spatial['terror_count'].min()) /\
                                    (regional_spatial['terror_count'].max() - regional_spatial['terror_count'].min())


regional_spatial_list = regional_spatial[['longitude', 'latitude', 'normalized_weight']].values.tolist()
regional_base_map = folium.Map(location=[30, 40], zoom_start=2)
HeatMap(regional_spatial_list).add_to(regional_base_map)
regional_base_map.save('regional.html')
regional_base_map

<h3>Insight #1 : The heatmap shows hotspots of deadly terrorist attacks (≥10 deaths) in South America since 2000, with clusters indicating regions of repeated severe violence. Normalized intensity highlights areas that may require focused security and policy intervention.</h3>

<h1>Crime Type Analysis</h1>

In [22]:
crime_fl1 = gt[(gt['alternative_txt'] == 'Insurgency/Guerilla Action') & (gt['iyear'] >= 2000) & (gt['nkill'] >= 20)]

crime = crime_fl1.groupby(['longitude', 'latitude']).size().reset_index(name='terror_count')
crime['normalized_weight'] = (crime['terror_count'] - crime['terror_count'].min()) /\
                                    (crime['terror_count'].max() - crime['terror_count'].min())


crime_list = crime[['longitude', 'latitude', 'normalized_weight']].values.tolist()
crime_bm = folium.Map(location=[30, 40], zoom_start=4)
HeatMap(crime_list).add_to(crime_bm)
crime_bm.save('crime.html')
crime_bm

<h3>Insight #2 : This heatmap highlights locations of deadly insurgency or guerilla attacks (≥20 deaths) worldwide since 2000, revealing clusters where such violence is most intense. Normalized density values emphasize areas with frequent and severe insurgent activity, providing a clear view for targeted policy or security responses.</h3>

<h1>Casualty Analysis</h1>

In [29]:
casualty = gt[
    (gt['iyear'] >= 2000) & 
    (gt['nkill'] >= 20) & 
    (gt['region_txt'] == 'Middle East & North Africa') &
    (gt['alternative_txt'] == 'Insurgency/Guerilla Action')]



casualty_spatial = casualty.groupby(['longitude', 'latitude']).size().reset_index(name='terror_count')
casualty_spatial['normalized_weight'] = (casualty_spatial['terror_count'] - casualty_spatial['terror_count'].min()) /\
                                    (casualty_spatial['terror_count'].max() - casualty_spatial['terror_count'].min())

casualty_sl = casualty_spatial[['longitude', 'latitude', 'normalized_weight']].values.tolist()
casualty_bm = folium.Map(location=[30, 40], zoom_start=2)
HeatMap(casualty_sl).add_to(casualty_bm)
casualty_bm.save('casualty.html')
casualty_bm

<h3>Insight #3 : This heatmap visualizes deadly terrorist attacks (≥20 deaths) since 2000 across the Middle East & North Africa with crime type of Insurgency/Guerilla Action. The highlighted hotspots indicate region with repeated high-casualty incidents, aiding in identifying areas most affected by extreme violence.</h3>

<h1>Demographic Analysis</h1>

In [34]:
demographic_f1 = gt[(gt['gname'] == 'Taliban') & (gt['weapsubtype1_txt'] == 'Landmine')]

demographic_s = demographic_f1.groupby(['longitude', 'latitude']).size().reset_index(name='terror_count')
demographic_s['normalized_weight'] = (demographic_s['terror_count'] - demographic_s['terror_count'].min()) /\
                                    (demographic_s['terror_count'].max() - demographic_s['terror_count'].min())

demographic_s_list = demographic_s[['longitude', 'latitude', 'normalized_weight']].values.tolist()
demo_base_map = folium.Map(location=[10, 30], zoom_start=2)
HeatMap(demographic_s_list).add_to(demo_base_map)
demo_base_map.save('demographic.html')
demo_base_map

<h3>Insight #4 : This heatmap displays the spatial distribution of Taliban attacks involving landmines, highlighting specific areas where this weapon type has been frequently used. The intensity of hotspots reveals zones with recurring landmine activity, useful for understanding patterns of deployment and assessing risk in affected regions.</h3>

<h1>Target Analysis</h1>

In [16]:
target = gt[(gt['targtype2_txt'] == 'Private Citizens & Property') & (gt['region_txt'] == 'Middle East & North Africa')]

target_s = target.groupby(['longitude', 'latitude']).size().reset_index(name='terror_count')
target_s['normalized_weight'] = (target_s['terror_count'] - target_s['terror_count'].min()) /\
                                    (target_s['terror_count'].max() - target_s['terror_count'].min())

target_s_list = target_s[['longitude', 'latitude', 'normalized_weight']].values.tolist()
target_bm = folium.Map(location=[10, 30], zoom_start=2)
HeatMap(target_s_list).add_to(target_bm)
target_bm.save('demographic.html')
target_bm

<h3>Insight #5 : This heatmap illustrates terrorist attacks targeting private citizens and property in the Middle East & North Africa, revealing concentrated areas of civilian-focused violence. The hotspots help identify regions where civilian populations face the greatest threat from recurring attacks.</h3>